In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
#     config = 'configs.config_WomenInEngineering_local'
    config = 'configs.config_ILLAE_local'

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### reading the dataset file ###

dfs=dict()
if c.directory['dataset']['type'] != 'xls':
    dfs['input'] = imp.pd.read_csv('{}data/raw/{}.{}'.format(c.directory['master'],
                                                     c.directory['dataset']['filename'],
                                                     c.directory['dataset']['type']
                                                    )
                           , sep=c.directory['dataset']['sep'])


imp.pprint(dfs['input'].shape)
util.display_df(dfs['input'].head(5))

(17354, 33)


In [4]:
### converting date ###

if c.date_format == 'timestamp':
#         util.init_parallel(False, c)
#         df_text['date_converted'] = df_text['date'].apply(str_timestamp_to_date)
#     dfs['input']['date_converted'] = util.parallelize_df_dask(dfs['input'][c.date_col], str_timestamp_to_date_df, c)    
    dfs['input']['date_converted'] = util.parallelize_df(dfs['input'][c.date_col], util.str_timestamp_to_date_df, c)    
#         df_text['date_converted'] = df_text['date'].apply(str_timestamp_to_date)    
    print('str_timestamp_to_date step is done!')
else:
    dfs['input']['date_converted'] = util.parallelize_df(dfs['input'][c.date_col], util.str_date_to_date_df, c)
#         df_text['date_converted'] = util.parallelize_df_dask(df_text['date'], str_date_to_date, c)        
    print('str_date_to_date step is done!')

imp.pprint('min date = {}'.format(min(dfs['input']['date_converted'])))
imp.pprint('max date = {}'.format(max(dfs['input']['date_converted'])))

parallel: 15 partitions with 15 cores for str_date_to_date_df
str_date_to_date step is done!
'min date = 2015-08-08 00:00:00'
'max date = 2015-10-15 00:00:00'


In [5]:
### filtering date and saving ###

min_date = imp.pd.to_datetime(c.min_date_val, infer_datetime_format=True)
max_date = imp.pd.to_datetime(c.max_date_val, infer_datetime_format=True)

if c.check_date:
    dfs['input'] = dfs['input'][(dfs['input']['date_converted'] <= max_date) & (dfs['input']['date_converted'] >= min_date)]

dfs['input'].to_csv('{}data/{}_1_data_filtered.csv'.format(c.directory['save'], c.project_name), index=False)

imp.pprint(dfs['input'].shape)

(17354, 34)
